In [189]:
SEASON = '1990-91'

In [190]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['zTotals'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK'])
p_df['zWS'] = zscore(p_df['WS'])
p_df['zVORP'] = zscore(p_df['VORP'])
p_df['zBPM'] = zscore(p_df['BPM'])
p_df['zTmsWins'] = zscore(p_df['TmsWins'])
p_df['RawScore'] = p_df['zTotals'] + p_df['zWS'] + p_df['zVORP'] + p_df['zBPM'] + p_df['zTmsWins']
p_df['PlayerScore'] = p_df['RawScore'] + c.PLAYER_SCORE_ADDITION # more positive PlayerScores
p_df['PlayerScore'] = p_df['PlayerScore'].apply(lambda x: pow(x, c.PLAYER_SCORE_EXPONENT) if x > 1 else x) # make higher PlayerScores more significant
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,DBPM,BPM,VORP,TmsWins,zTotals,zWS,zVORP,zBPM,zTmsWins,RawScore
1,Michael Jordan,CHI,SG,133.799061,27,82,82,37.0,12.1,22.4,...,3.2,12.0,10.8,61.0,4.303564,4.756141,5.360730,4.147849,1.591573,20.159858
2,David Robinson,SAS,C,103.984400,25,82,81,37.7,9.2,16.7,...,3.3,8.5,8.2,55.0,4.272123,3.792983,3.938601,2.999195,1.109985,16.112888
3,Magic Johnson,LAL,PG,96.489342,31,79,79,37.1,5.9,12.4,...,1.9,9.0,8.1,58.0,3.313149,3.325997,3.883904,3.163289,1.350779,15.037117
4,John Stockton,UTA,PG,86.813337,28,82,82,37.8,6.0,11.9,...,2.5,8.3,8.1,54.0,2.841522,2.917384,3.883904,2.933558,1.029721,13.606089
5,Karl Malone,UTA,PF,85.688322,27,82,82,40.3,10.3,19.6,...,0.6,5.4,6.2,54.0,4.224960,3.355183,2.844656,1.981817,1.029721,13.436337
6,Charles Barkley,PHI,SF,85.398053,27,67,67,37.3,9.9,17.4,...,1.0,9.3,7.0,44.0,3.879101,2.742264,3.282234,3.261745,0.227075,13.392419
7,Clyde Drexler,POR,SG,78.472771,28,82,82,34.8,7.9,16.3,...,1.7,6.8,6.4,63.0,2.731476,2.450398,2.954051,2.441278,1.752102,12.329305
8,Kevin Johnson,PHO,PG,74.432833,24,77,76,36.0,7.7,14.9,...,1.1,6.6,6.0,55.0,2.935847,2.537958,2.735262,2.375641,1.109985,11.694693
9,Terry Porter,POR,PG,74.043702,27,81,81,32.9,6.0,11.7,...,1.8,7.4,6.2,63.0,1.772501,2.625518,2.844656,2.638190,1.752102,11.632968
10,Scottie Pippen,CHI,SF,69.587291,25,82,82,36.8,7.3,14.1,...,2.5,5.8,5.9,61.0,2.432779,2.100159,2.680565,2.113091,1.591573,10.918166
